# Notebook KPIs
En este notebook se realizaran los distintos set de datos que serán usados para medir los KPI propuestos, los cuales son los siguientes:<br><br>

1. Reducir en un 10% la **tasa de homicidios** en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.  
    - **Tasa de homicidios en siniestros viales =** (Número de homicidios en siniestros viales / Población total) * 100,000  <br><br>

2. Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.  <br><br>

3. El 3° KPI es de libre elección, he optado por centrarnos en reducir la principal victima de accidentes en la vía, como vimos en el EDA, estos son los Peatones, por lo que planteo como un KPI razonable una disminución del 10% en la cantidad de accidentes mortales de peatones en cada año, respecto al anterior.

# Importamos las librerías necesarias y los datos que usaremos

In [40]:
import pandas as pd # Trabajaremos los datos con pandas.

In [41]:
df = pd.read_csv('../Data/Processed/homicidios_clean.csv', encoding='utf-8')

# KPI1: Reducir en un 10% la tasa de homicidios en siniestro viales de los últimos seis meses, respecto al semestre anterior.

Para calcular esta taza, necesitamos saber la población total de la ciudad de Buenos Aires, el censo 2022 indica 3,121,707 habitantes, y el censo 2010 indica 2,890,151 habitantes, al ser los únicos censos disponibles, debemos calcular el aproximado de población de cada semestre que evaluaremos.

In [42]:
c10 = 2_890_151
c22 = 3_121_707
# Hacemos una regla de 3 para hallar el valor aproximado de crecimiento por semestre desde el 2010 hasta el 2022.
crece = (c22 - c10)/10 # El crecimiento total en 10 años es el censo de 2022 menos el de 2010. Esto lo dividimos sobre la cantidad de años que pasaron. En este caso 10, para obtener el crecimiento de cada año.
print(f"Cada año entre 2010 y 2022, la población creció en {crece} habitantes")
# Obtenemos el aproximado de cada año de nuestro DataFrame y lo guardamos en un diccionario.
print()
pob = {}
an = 2010 # El año desde que empezaremos a medir.
pb = c10 # La población desde la que empezaremos a medir.
for anio in df['Anio'].unique():
    # Solo tomaremos los años que estén dentro de nuestro DataFrame, por lo que mientras que el año actual no se encuentre, se le sumará el crecimiento calculado previamente, y 1 año al año actual evaluando.
    while an < anio:
        pb += crece
        an += 1
    pob[anio] = int(pb) # Definimos la población para el año actual, en un número entero.
pob = pd.DataFrame(list(pob.items()), columns=['Anio', 'Población']).set_index('Anio')
pob

Cada año entre 2010 y 2022, la población creció en 23155.6 habitantes



,Población
Anio,
2016,3029084
2017,3052240
2018,3075395
2019,3098551
2020,3121707
2021,3144862


Ahora agrupemos los datos para contar las victimas en cada semestre.

In [43]:
df['Fecha'] = pd.to_datetime(df['Anio'].astype(str) + '-' + df['Mes'].astype(str)) # Relacionamos el mes y el año.
semestre = df.groupby(pd.Grouper(key='Fecha', freq='6ME'))['NVictimas'].sum().reset_index() # Obtenemos las victimas por semestre.
df.drop(columns=['Fecha']) # Eliminamos la columna recién creada.
semestre['Fecha'] = semestre['Fecha'].dt.strftime('%Y-%m') # Convertimos la fecha a mes y año.
semestre # Mostramos el resultado.

,Fecha,NVictimas
0,2016-01,7
1,2016-07,64
2,2017-01,70
3,2017-07,61
4,2018-01,71
5,2018-07,69
6,2019-01,72
7,2019-07,50
8,2020-01,49
9,2020-07,24


corregimos el último semestre ya que no hay registros de 2022.

In [44]:
semestre.loc[12] = {'Fecha': '2021-12', 'NVictimas':35}

Creamos la formula para calcular la tasa de homicidios en siniestros viales

In [45]:
def get_tasa(hom, tot):
    return (hom/tot) * 100_000

Aplicamos la tasa a el DataFrame

In [46]:
semestre['Poblacion'] = semestre['Fecha'].apply(lambda x: pob.loc[int(x[:4])].iloc[0]) # Identificamos la población de cada semestre.
semestre['TasaHomicidios'] = semestre.apply(lambda row: get_tasa(row['NVictimas'], row['Poblacion']), axis=1) # Aplicamos la formula a las victimas y al total de población.

Ahora calculamos el cambio porcentual que hubo respecto al anterior.

In [47]:
semestre['Cambio'] = round(semestre['TasaHomicidios'].pct_change() * 100, 2)

In [48]:
semestre

,Fecha,NVictimas,Poblacion,TasaHomicidios,Cambio
0,2016-01,7,3029084,0.231093,NaN
1,2016-07,64,3029084,2.112850,814.29
2,2017-01,70,3052240,2.293398,8.55
3,2017-07,61,3052240,1.998532,-12.86
4,2018-01,71,3075395,2.308647,15.52
5,2018-07,69,3075395,2.243614,-2.82
6,2019-01,72,3098551,2.323667,3.57
7,2019-07,50,3098551,1.613657,-30.56
8,2020-01,49,3121707,1.569654,-2.73
9,2020-07,24,3121707,0.768810,-51.02


Observando el resultado, eliminaré el primer semestre (2016-01) debido a que puede ser considerado un outlier, que alterará los resultados de nuestro análisis.

In [49]:
semestre = semestre.drop(0).reset_index(drop=True)

Corregimos el primer semestre para que sea 0 el cambio.

In [50]:
semestre.loc[0, 'Cambio'] = 0

Eliminamos las columnas de sobra.

In [51]:
semestre.drop(columns=['NVictimas', 'Poblacion'], inplace=True)

Observamos y exportamos el resultado final.

In [52]:
semestre

,Fecha,TasaHomicidios,Cambio
0,2016-07,2.112850,0.00
1,2017-01,2.293398,8.55
2,2017-07,1.998532,-12.86
3,2018-01,2.308647,15.52
4,2018-07,2.243614,-2.82
5,2019-01,2.323667,3.57
6,2019-07,1.613657,-30.56
7,2020-01,1.569654,-2.73
8,2020-07,0.768810,-51.02
9,2021-01,1.907874,148.16


In [53]:
semestre.rename(columns={'Fecha':'Semestre'}, inplace=True)

In [54]:
semestre.to_csv('../Data/KPIs/kpi_homicidios.csv', index=False)

# KPI 2: Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, respecto al año anterior.

Filtramos los datos para obtener los accidentes en moto.

In [55]:
df_moto = df[df['Victima'] == 'moto']

Contamos la cantidad de accidentes que hubo en moto para cada año.

In [56]:
df_moto = df_moto.groupby('Anio').size().reset_index()

Nombramos la columna de accidentes totales.

In [57]:
df_moto.rename(columns={0:'Accidentes'}, inplace=True)

Calculamos el cambio porcentual que hubo respecto al año anterior.

In [58]:
df_moto['Cambio'] = round(df_moto['Accidentes'].pct_change() * 100, 2)

Corregimos la primera columna para que sea un 0 y no un NAN.

In [59]:
df_moto.loc[0, 'Cambio'] = 0

Mostramos y exportamos el resultado.

In [60]:
df_moto

,Anio,Accidentes,Cambio
0,2016,59,0.00
1,2017,47,-20.34
2,2018,55,17.02
3,2019,47,-14.55
4,2020,26,-44.68
5,2021,45,73.08


In [61]:
df_moto.to_csv('../Data/KPIs/kpi_motos.csv', index=False)

# KPI 3: Reducir en un 10% la cantidad de accidentes mortales de peatones en el último año, respecto al año anterior.

Repetimos el proceso para crear el anterior KPI, pero esta vez con peatones.

In [62]:
df_pea = df[df['Victima'] == 'peaton']
df_pea = df_pea.groupby('Anio').size().reset_index()
df_pea.rename(columns={0:'Accidentes'}, inplace=True)
df_pea['Cambio'] = round(df_pea['Accidentes'].pct_change() * 100, 2)
df_pea.loc[0, 'Cambio'] = 0

In [63]:
df_pea

,Anio,Accidentes,Cambio
0,2016,46,0.00
1,2017,49,6.52
2,2018,63,28.57
3,2019,34,-46.03
4,2020,34,0.00
5,2021,33,-2.94


In [64]:
df_pea.to_csv('../Data/KPIs/kpi_peatones.csv', index=False)